In [1]:
import tensorflow as tf
import numpy as np

### 定义TextRNN结构，使用双向的LSTM

In [2]:
class TextRNN:
    def __init__(self, batch_size, num_classes, vocab_size, sentence_len, embed_size, 
                 learning_rate, decay_steps, decay_rate, is_training):
        #1.定义超参数
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.sentence_len = sentence_len
        self.embed_size = embed_size
        self.hidden_size = embed_size #lstm层的维度
        self.learning_rate = learning_rate
        self.is_training = is_training
        self.initializer = tf.random_normal_initializer(stddev=0.1)
        
        #epoch信息
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.epoch_step = tf.Variable(0, trainable=False, name='epoch_step')
        self.epoch_increment = tf.assign(self.epoch_step, tf.add(self.epoch_step, tf.constant(1)))
        self.decay_steps, self.decay_rate = decay_steps, decay_rate
        
        #2.输入
        self.input_x = tf.placeholder(tf.int32, [None, sentence_len], 'input_x')
#         self.input_y = tf.placeholder(tf.int32, [None], 'input_y') #单个标签
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], 'input_y') #多个标签
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        #3.初始化全连接层参数
        self.init_weight()
        
        #4.网络结构
        self.logits = self.inference() #[batch_size, num_classes]
        
        #5.损失函数
        self.loss_val = self.loss()
        
        #6.优化器
        self.train_op = self.train()
        
    def init_weight(self):
        self.Embedding = tf.get_variable('Embedding', [self.vocab_size, self.embed_size], dtype=tf.float32)
        self.W = tf.get_variable('W', [self.hidden_size * 2, self.num_classes], dtype=tf.float32) #双向LSTM，输出concat，所以此处为2倍
        self.b = tf.get_variable('b', [self.num_classes], dtype=tf.float32)
        
    def inference(self):
        # a.embedding
        self.sentence_embed = tf.nn.embedding_lookup(self.Embedding, self.input_x) #[batch_size, sentence_len, embed_size]
        
        # b.bidiretional lstm
        self.fw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #前向单元
        self.bw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #后向单元
#         if self.dropout_keep_prob is not None:
#             self.fw_cell = tf.contrib.rnn.DropoutWrapper(self.fw_cell, output_keep_prob=) 
                #input_keep_prob是对输入而言，output_keep_prb是对lstm各层而言
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(self.fw_cell, self.bw_cell, self.sentence_embed, dtype=tf.float32)
        #输入为 [batch_size, sentence_len, embed_size]，输出为大小为2的元组，每个元素为[batch_size, sentence_len, hidden_size]
        
        # c.concat
        fw_output = outputs[0][:,-1,:]
        bw_output = outputs[1][:,-1,:] #[batch_size, 1, hidden_size]
        final_output = tf.concat([fw_output, bw_output], axis=1) #[batch_size, 1, hidden_size*2]
        final_output = tf.reshape(final_output, [-1, self.hidden_size*2]) #[batch_size, hidden_size * 2]
        
        # d.full_connection
        logits = tf.matmul(final_output, self.W) + self.b #[batch_size, num_classes]
        return logits
    
    def loss(self, l2_lambda=0.0001):
#         loss1 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
        #先将label转化为one-hot形式，再对logits计算softmax，最后计算交叉熵
        loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
        loss1 = tf.reduce_mean(tf.reduce_sum(loss1, axis=1))
        loss2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()]) * l2_lambda
        return loss1 + loss2
    
    def train(self):
        learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps, self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, self.global_step, learning_rate, optimizer='Adam')
        return train_op
    

### 测试

In [3]:
def test():
    num_classes=5
    learning_rate=0.01
    batch_size=5
    decay_step=1000
    decay_rate=0.9
    sequence_length=5
    vocab_size=10000
    embed_size=100
    is_training=True
    dropout_keep_prob=0.5
    
    model = TextRNN(batch_size, num_classes, vocab_size, sequence_length, embed_size, 
                     learning_rate, decay_step, decay_rate, True)
    print(tf.trainable_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        input_x = np.random.randint(0,100,size=(batch_size, sequence_length),dtype=np.int32)
        input_y = np.random.randint(0, 2,size=(batch_size, num_classes), dtype=np.int32)
        for i in range(20):
            #input_x = np.zeros((batch_size, sequence_length), dtype=np.int32)
            #input_y = np.array([1,0,1,1,1,2,1,1], dtype=np.int32)
            loss, logits, _ = sess.run([model.loss_val, model.logits, model.train_op],
                                            feed_dict={model.input_x: input_x, model.input_y: input_y,
                                                       model.dropout_keep_prob: dropout_keep_prob})
            logits = np.argsort(logits)
            print('****label****\n', input_y)
            print('****pre_y****\n', logits, '\n')

In [4]:
tf.reset_default_graph()
test()

[<tf.Variable 'Embedding:0' shape=(10000, 100) dtype=float32_ref>, <tf.Variable 'W:0' shape=(200, 5) dtype=float32_ref>, <tf.Variable 'b:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>]
****label****
 [[1 1 1 1 0]
 [1 0 1 1 0]
 [1 0 1 0 1]
 [1 1 0 0 1]
 [1 1 0 0 1]]
****pre_y****
 [[1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]] 

****label****
 [[1 1 1 1 0]
 [1 0 1 1 0]
 [1 0 1 0 1]
 [1 1 0 0 1]
 [1 1 0 0 1]]
****pre_y****
 [[1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]
 [1 0 3 2 4]] 

****label****
 [[1 1 1 1 0]
 [1 0 1 1 0]
 [1 0 1 0 1]
 [1 1 0 0 1]
 [1 1 0 0 1]]
****pre_y****
 [[1 3 0 4 2]
 [1 3 0 2 4]

### 训练

In [5]:
import sys
import tensorflow as tf
import numpy as np
# from tflearn.data_utils import to_categorical, pad_sequences
import os
import pickle
import h5py

In [6]:
#定义超参数
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('batch_size', 128, 'batch_size')
tf.app.flags.DEFINE_integer('num_classes', 1999, 'num_classes')
tf.app.flags.DEFINE_integer('sentence_len', 100, 'length of each sentence')
tf.app.flags.DEFINE_integer('embed_size', 100, 'embedding size')
tf.app.flags.DEFINE_float('learning_rate', 0.01, '')
tf.app.flags.DEFINE_float('decay_rate', 0.8, '')
tf.app.flags.DEFINE_integer('decay_steps', 10000, 'number of steps before decay learning rate')
tf.app.flags.DEFINE_bool('is_training', True, '')

tf.app.flags.DEFINE_integer('num_epoch', 10, 'number of epoch')

tf.app.flags.DEFINE_string("ckpt_dir","testrnn_multilabel_small_checkpoint/","checkpoint location for the model")
tf.app.flags.DEFINE_string("cache_path","textrnn_multilabel_small_checkpoint/data_cache.pik","data chche for the model")

In [7]:
import time
def log(str):
    t = time.localtime()
    print("[%4d/%02d/%02d %02d:%02d:%02d]"%(t.tm_year, t.tm_mon, t.tm_mday, t.tm_hour, t.tm_min, t.tm_sec), end=' ')
    print(str)

In [8]:
def main(_):
    #1.加载数据
    base_path = '/data/chenhy/data/ieee_zhihu_cup/'
    cache_file_h5py = base_path + 'data.h5'
    cache_file_pickle = base_path + 'vocab_label.pik'
    word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y = load_data(cache_file_h5py, cache_file_pickle)
    vocab_size = len(word2index)
    index2word = {index: word for word, index in word2index.items()}
    index2label = {index: label for label, index in label2index.items()}

    print("train_X[0:5]:", train_X[0:5])
    print("train_Y[0:5]:", train_y[0:5])
    train_y_short = get_target_label_short(train_y[0])
    print("train_y_short:", train_y_short)
    
    #2.创建session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        model = TextRNN(FLAGS.batch_size, FLAGS.num_classes, vocab_size, FLAGS.sentence_len,FLAGS.embed_size, 
                        FLAGS.learning_rate, FLAGS.decay_steps, FLAGS.decay_rate, FLAGS.is_training)
        saver = tf.train.Saver()
        batch_size = FLAGS.batch_size
        CONTINUE_TRAIN = True
        if os.path.exists(FLAGS.ckpt_dir + 'checkpoint'):
            log("restore from checkpoint")
            saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
            if CONTINUE_TRAIN: log("continue training...")
        else:
            log('init variables')
            sess.run(tf.global_variables_initializer())
#             #是否使用embedding
#             print('assign pre-trained embedding')
#             embedding_assign = tf.assign(model.Embedding, tf.constant(np.array(embedding_final))) #为model.Embedding赋值
#             sess.run(embedding_assign)
        if not os.path.exists(FLAGS.ckpt_dir + 'checkpoint') or CONTINUE_TRAIN:
            num_of_data = len(train_y)
            for _ in range(FLAGS.num_epoch):
                epoch = sess.run(model.epoch_step)
                loss, counter = 0., 0.
                for start, end in zip(range(0, num_of_data, batch_size), range(batch_size, num_of_data, batch_size)):
                    loss_tmp,  _ = sess.run([model.loss_val, model.train_op], 
                                                    feed_dict={model.input_x: train_X[start:end, :FLAGS.sentence_len], model.input_y: train_y[start:end],
                                                               model.dropout_keep_prob: 1})
                    loss, counter = loss + loss_tmp, counter + 1
                    if counter % 200 == 0:
                        log("Epoch %d\Batch %d\ Train Loss:%.3f"%(epoch, counter, loss/float(counter)))
                    if counter % 1400 == 0:
                        print('run model on validation data...')
                        loss_valid, f1_score, precision, recall = do_eval(sess, model, vaild_X, valid_y)
                        log("Epoch %d/ Validation Loss:%.3f/ F1_score:%.3f/ Precision:%.3f/ Recall:%.3f"%(epoch, loss_valid, f1_score, precision, recall))
                        #save the checkpoint
                        save_path = FLAGS.ckpt_dir + 'model.ckpt'
                        saver.save(sess, save_path, global_step=model.epoch_step)
                sess.run(model.epoch_increment)
            
    

In [9]:
#一些辅助函数
def get_target_label_short(eval_y):
    res = [idx for idx in range(len(eval_y)) if eval_y[idx] > 0] #结果如：[45,100,1555]
    return res

def get_label_using_logits(logits, top_number=5):
    predict_y = [idx for idx in range(len(logits)) if logits[idx] >= 0.5]
    if len(predict_y) == 0: predict_y = [np.argmax(logits)]
    return predict_y

def load_data(h5_file_path, pik_file_path):
    if not os.path.exists(h5_file_path) or not os.path.exists(pik_file_path):
        raise RuntimeError('No such file!!')

    print('cache files exist, going to load in...')
    print('loading h5_file...')
    h5_file = h5py.File(h5_file_path, 'r')
    print('h5_file.keys:', h5_file.keys())
    train_X, train_y = h5_file['train_X'], h5_file['train_Y']
    vaild_X, valid_y = h5_file['vaild_X'], h5_file['valid_Y']
    test_X,  test_y  = h5_file['test_X'],  h5_file['test_Y']
    #embedding_final = h5_file['embedding']

    print('loading pickle file')
    word2index, label2index = None, None
    with open(pik_file_path, 'rb') as pkl:
        word2index,label2index = pickle.load(pkl)
    print('cache files load successful!')
    return word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y

def do_eval(sess, model, eval_X, eval_y):
    test_X, test_y = eval_X[:3000], eval_y[:3000]
    num_of_data = len(test_y)
    batch_size = 1
    loss, F1, p, r = 0., 0., 0., 0.
    label_dict_confuse = {'TP':0.000001, 'FN':0.000001, 'FP':0.000001}
    for start in range(num_of_data):
        end = start + 1
        lo,logits = sess.run([model.loss_val, model.logits], 
                        feed_dict={model.input_x: test_X[start:end,:FLAGS.sentence_len], model.input_y: test_y[start:end],
                                   model.dropout_keep_prob:1.0})
        loss += lo
        pre = get_label_using_logits(logits[0])
        label = get_target_label_short(test_y[start])
#         pre = np.argsort(logits[0])[-5:]
#         label = [i for i in range(len(test_y[start])) if test_y[start][i] > 0]
        if start == 0: print('label:',label, 'predict:', pre)
        inter = len([x for x in pre if x in label])
        label_dict_confuse['TP'] += inter
        label_dict_confuse['FN'] += len(label) - inter
        label_dict_confuse['FP'] += len(pre) - inter
    print(label_dict_confuse)
    p = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FP'])
    r = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FN'])
    if p + r == 0: return loss/num_of_data, 0, 0, 0
    F1 = (2 * p * r)/(p + r)
    return loss/num_of_data, F1, p, r


In [11]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X[0:5]: [[ 832   60  256 1172 3407  516   96  138  103 1108   16    3   96  177
    22   11  672   53   18 1560 1560   15   65   12  180   10  342  173
    13  103  141  707  191   12  342  173   15   13   22   11  229  264
   163 1362  135 1249  156  156  731  115   84   10  808 1713  103  141
   229  264  788  421  103  141   12   95  316   10  808 1713  103  141
    12 2413 1227   15 1397  997   22  116  301  489   12   18  858   99
   596   98   26  646  813   10  386 1093  197  767   22   11 1179 1849
   593   84   22   11   94  102  322   60  190  220  583  355   10  153
   103  141  192  153   12   10 1244  466  116  103  141  189   58  130
    95  316   12  788  421  699   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 

[2019/03/24 14:38:23] Epoch 0\Batch 6200\ Train Loss:20.271
[2019/03/24 14:38:51] Epoch 0\Batch 6400\ Train Loss:20.229
[2019/03/24 14:39:20] Epoch 0\Batch 6600\ Train Loss:20.192
[2019/03/24 14:39:49] Epoch 0\Batch 6800\ Train Loss:20.154
[2019/03/24 14:40:18] Epoch 0\Batch 7000\ Train Loss:20.119
run model on validation data...
label: [572, 793] predict: [0]
{'FP': 2966.000001, 'TP': 34.000001, 'FN': 6965.000001}
[2019/03/24 14:40:56] Epoch 0/ Validation Loss:18.838/ F1_score:0.007/ Precision:0.011/ Recall:0.005
[2019/03/24 14:41:25] Epoch 0\Batch 7200\ Train Loss:20.083
[2019/03/24 14:41:54] Epoch 0\Batch 7400\ Train Loss:20.051
[2019/03/24 14:42:22] Epoch 0\Batch 7600\ Train Loss:20.019
[2019/03/24 14:42:51] Epoch 0\Batch 7800\ Train Loss:19.991
[2019/03/24 14:43:20] Epoch 0\Batch 8000\ Train Loss:19.960
[2019/03/24 14:43:49] Epoch 0\Batch 8200\ Train Loss:19.931
[2019/03/24 14:44:16] Epoch 0\Batch 8400\ Train Loss:19.903
run model on validation data...
label: [572, 793] predict: [

[2019/03/24 15:30:00] Epoch 1\Batch 1400\ Train Loss:12.785
run model on validation data...
label: [572, 793] predict: [23]
{'FP': 2024.0000009999999, 'TP': 1014.000001, 'FN': 5985.000001}
[2019/03/24 15:30:39] Epoch 1/ Validation Loss:12.695/ F1_score:0.202/ Precision:0.334/ Recall:0.145
[2019/03/24 15:31:08] Epoch 1\Batch 1600\ Train Loss:12.770
[2019/03/24 15:31:37] Epoch 1\Batch 1800\ Train Loss:12.749
[2019/03/24 15:32:06] Epoch 1\Batch 2000\ Train Loss:12.740
[2019/03/24 15:32:35] Epoch 1\Batch 2200\ Train Loss:12.733
[2019/03/24 15:33:04] Epoch 1\Batch 2400\ Train Loss:12.724
[2019/03/24 15:33:32] Epoch 1\Batch 2600\ Train Loss:12.714
[2019/03/24 15:34:01] Epoch 1\Batch 2800\ Train Loss:12.707
run model on validation data...
label: [572, 793] predict: [23]
{'FP': 1982.0000009999999, 'TP': 1046.0000009999999, 'FN': 5953.000001}
[2019/03/24 15:34:40] Epoch 1/ Validation Loss:12.593/ F1_score:0.209/ Precision:0.345/ Recall:0.149
[2019/03/24 15:35:08] Epoch 1\Batch 3000\ Train Loss:

[2019/03/24 16:20:10] Epoch 1\Batch 18400\ Train Loss:12.353
[2019/03/24 16:20:38] Epoch 1\Batch 18600\ Train Loss:12.349
[2019/03/24 16:21:06] Epoch 1\Batch 18800\ Train Loss:12.344
[2019/03/24 16:21:35] Epoch 1\Batch 19000\ Train Loss:12.341
[2019/03/24 16:22:03] Epoch 1\Batch 19200\ Train Loss:12.337
[2019/03/24 16:22:32] Epoch 1\Batch 19400\ Train Loss:12.333
[2019/03/24 16:23:00] Epoch 1\Batch 19600\ Train Loss:12.329
run model on validation data...
label: [572, 793] predict: [23]
{'FP': 1864.0000009999999, 'TP': 1198.0000009999999, 'FN': 5801.000001}
[2019/03/24 16:23:38] Epoch 1/ Validation Loss:11.942/ F1_score:0.238/ Precision:0.391/ Recall:0.171
[2019/03/24 16:24:07] Epoch 1\Batch 19800\ Train Loss:12.326
[2019/03/24 16:24:35] Epoch 1\Batch 20000\ Train Loss:12.322
[2019/03/24 16:25:03] Epoch 1\Batch 20200\ Train Loss:12.318
[2019/03/24 16:25:31] Epoch 1\Batch 20400\ Train Loss:12.315
[2019/03/24 16:25:59] Epoch 1\Batch 20600\ Train Loss:12.311
[2019/03/24 16:26:26] Epoch 1\B

[2019/03/24 17:12:00] Epoch 2\Batch 13600\ Train Loss:11.758
[2019/03/24 17:12:27] Epoch 2\Batch 13800\ Train Loss:11.756
[2019/03/24 17:12:55] Epoch 2\Batch 14000\ Train Loss:11.753
run model on validation data...
label: [572, 793] predict: [23]
{'FP': 1785.0000009999999, 'TP': 1290.0000009999999, 'FN': 5709.000001}
[2019/03/24 17:13:36] Epoch 2/ Validation Loss:11.499/ F1_score:0.256/ Precision:0.420/ Recall:0.184
[2019/03/24 17:14:03] Epoch 2\Batch 14200\ Train Loss:11.749
[2019/03/24 17:14:31] Epoch 2\Batch 14400\ Train Loss:11.746
[2019/03/24 17:14:58] Epoch 2\Batch 14600\ Train Loss:11.743
[2019/03/24 17:15:26] Epoch 2\Batch 14800\ Train Loss:11.739
[2019/03/24 17:15:55] Epoch 2\Batch 15000\ Train Loss:11.737
[2019/03/24 17:16:24] Epoch 2\Batch 15200\ Train Loss:11.733
[2019/03/24 17:16:53] Epoch 2\Batch 15400\ Train Loss:11.730
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1794.0000009999999, 'TP': 1285.0000009999999, 'FN': 5714.000001}
[2019/03/24 17:1

...

[2019/03/24 23:12:30] Epoch 7\Batch 21600\ Train Loss:10.263
[2019/03/24 23:12:59] Epoch 7\Batch 21800\ Train Loss:10.263
[2019/03/24 23:13:28] Epoch 7\Batch 22000\ Train Loss:10.262
[2019/03/24 23:13:57] Epoch 7\Batch 22200\ Train Loss:10.261
[2019/03/24 23:14:26] Epoch 7\Batch 22400\ Train Loss:10.261
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1602.0000009999999, 'TP': 1485.0000009999999, 'FN': 5514.000001}
[2019/03/24 23:15:13] Epoch 7/ Validation Loss:10.277/ F1_score:0.294/ Precision:0.481/ Recall:0.212
[2019/03/24 23:15:42] Epoch 7\Batch 22600\ Train Loss:10.261
[2019/03/24 23:16:11] Epoch 7\Batch 22800\ Train Loss:10.260
[2019/03/24 23:16:40] Epoch 7\Batch 23000\ Train Loss:10.260
[2019/03/24 23:17:27] Epoch 8\Batch 200\ Train Loss:10.271
[2019/03/24 23:17:57] Epoch 8\Batch 400\ Train Loss:10.231
[2019/03/24 23:18:26] Epoch 8\Batch 600\ Train Loss:10.217
[2019/03/24 23:18:55] Epoch 8\Batch 800\ Train Loss:10.208
[2019/03/24 23:19:24] Epoch 8\Batch 10

[2019/03/25 00:05:22] Epoch 8\Batch 16400\ Train Loss:10.190
[2019/03/25 00:05:51] Epoch 8\Batch 16600\ Train Loss:10.189
[2019/03/25 00:06:20] Epoch 8\Batch 16800\ Train Loss:10.189
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1591.0000009999999, 'TP': 1501.0000009999999, 'FN': 5498.000001}
[2019/03/25 00:07:07] Epoch 8/ Validation Loss:10.242/ F1_score:0.297/ Precision:0.485/ Recall:0.214
[2019/03/25 00:07:36] Epoch 8\Batch 17000\ Train Loss:10.188
[2019/03/25 00:08:05] Epoch 8\Batch 17200\ Train Loss:10.188
[2019/03/25 00:08:34] Epoch 8\Batch 17400\ Train Loss:10.189
[2019/03/25 00:09:03] Epoch 8\Batch 17600\ Train Loss:10.189
[2019/03/25 00:09:32] Epoch 8\Batch 17800\ Train Loss:10.188
[2019/03/25 00:10:01] Epoch 8\Batch 18000\ Train Loss:10.187
[2019/03/25 00:10:31] Epoch 8\Batch 18200\ Train Loss:10.187
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1588.0000009999999, 'TP': 1502.0000009999999, 'FN': 5497.000001}
[2019/03/25 00:

[2019/03/25 00:58:13] Epoch 9\Batch 11200\ Train Loss:10.141
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1597.0000009999999, 'TP': 1492.0000009999999, 'FN': 5507.000001}
[2019/03/25 00:59:00] Epoch 9/ Validation Loss:10.227/ F1_score:0.296/ Precision:0.483/ Recall:0.213
[2019/03/25 00:59:29] Epoch 9\Batch 11400\ Train Loss:10.141
[2019/03/25 00:59:58] Epoch 9\Batch 11600\ Train Loss:10.141
[2019/03/25 01:00:27] Epoch 9\Batch 11800\ Train Loss:10.141
[2019/03/25 01:00:56] Epoch 9\Batch 12000\ Train Loss:10.141
[2019/03/25 01:01:25] Epoch 9\Batch 12200\ Train Loss:10.139
[2019/03/25 01:01:55] Epoch 9\Batch 12400\ Train Loss:10.138
[2019/03/25 01:02:24] Epoch 9\Batch 12600\ Train Loss:10.137
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1596.0000009999999, 'TP': 1490.0000009999999, 'FN': 5509.000001}
[2019/03/25 01:03:10] Epoch 9/ Validation Loss:10.222/ F1_score:0.295/ Precision:0.483/ Recall:0.213
[2019/03/25 01:03:40] Epoch 9\Batch 

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X[0:5]: [[ 832   60  256 1172 3407  516   96  138  103 1108   16    3   96  177
    22   11  672   53   18 1560 1560   15   65   12  180   10  342  173
    13  103  141  707  191   12  342  173   15   13   22   11  229  264
   163 1362  135 1249  156  156  731  115   84   10  808 1713  103  141
   229  264  788  421  103  141   12   95  316   10  808 1713  103  141
    12 2413 1227   15 1397  997   22  116  301  489   12   18  858   99
   596   98   26  646  813   10  386 1093  197  767   22   11 1179 1849
   593   84   22   11   94  102  322   60  190  220  583  355   10  153
   103  141  192  153   12   10 1244  466  116  103  141  189   58  130
    95  316   12  788  421  699   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 

[2019/03/25 08:42:14] Epoch 9\Batch 5800\ Train Loss:10.108
[2019/03/25 08:42:43] Epoch 9\Batch 6000\ Train Loss:10.108
[2019/03/25 08:43:12] Epoch 9\Batch 6200\ Train Loss:10.108
[2019/03/25 08:43:41] Epoch 9\Batch 6400\ Train Loss:10.106
[2019/03/25 08:44:10] Epoch 9\Batch 6600\ Train Loss:10.105
[2019/03/25 08:44:39] Epoch 9\Batch 6800\ Train Loss:10.107
[2019/03/25 08:45:08] Epoch 9\Batch 7000\ Train Loss:10.107
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1595.0000009999999, 'TP': 1495.0000009999999, 'FN': 5504.000001}
[2019/03/25 08:45:54] Epoch 9/ Validation Loss:10.208/ F1_score:0.296/ Precision:0.484/ Recall:0.214
[2019/03/25 08:46:24] Epoch 9\Batch 7200\ Train Loss:10.107
[2019/03/25 08:46:53] Epoch 9\Batch 7400\ Train Loss:10.108
[2019/03/25 08:47:22] Epoch 9\Batch 7600\ Train Loss:10.108
[2019/03/25 08:47:51] Epoch 9\Batch 7800\ Train Loss:10.108
[2019/03/25 08:48:20] Epoch 9\Batch 8000\ Train Loss:10.106
[2019/03/25 08:48:49] Epoch 9\Batch 8200\ 

[2019/03/25 09:34:30] Epoch 10\Batch 400\ Train Loss:10.103
[2019/03/25 09:34:59] Epoch 10\Batch 600\ Train Loss:10.089
[2019/03/25 09:35:28] Epoch 10\Batch 800\ Train Loss:10.078
[2019/03/25 09:35:58] Epoch 10\Batch 1000\ Train Loss:10.082
[2019/03/25 09:36:27] Epoch 10\Batch 1200\ Train Loss:10.090
[2019/03/25 09:36:55] Epoch 10\Batch 1400\ Train Loss:10.091
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1579.0000009999999, 'TP': 1511.0000009999999, 'FN': 5488.000001}
[2019/03/25 09:37:42] Epoch 10/ Validation Loss:10.196/ F1_score:0.300/ Precision:0.489/ Recall:0.216
[2019/03/25 09:38:11] Epoch 10\Batch 1600\ Train Loss:10.087
[2019/03/25 09:38:40] Epoch 10\Batch 1800\ Train Loss:10.077
[2019/03/25 09:39:09] Epoch 10\Batch 2000\ Train Loss:10.081
[2019/03/25 09:39:38] Epoch 10\Batch 2200\ Train Loss:10.084
[2019/03/25 09:40:07] Epoch 10\Batch 2400\ Train Loss:10.082
[2019/03/25 09:40:36] Epoch 10\Batch 2600\ Train Loss:10.081
[2019/03/25 09:41:05] Epoch 10\B

[2019/03/25 10:25:57] Epoch 10\Batch 17800\ Train Loss:10.076
[2019/03/25 10:26:26] Epoch 10\Batch 18000\ Train Loss:10.075
[2019/03/25 10:26:55] Epoch 10\Batch 18200\ Train Loss:10.075
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1588.0000009999999, 'TP': 1500.0000009999999, 'FN': 5499.000001}
[2019/03/25 10:27:41] Epoch 10/ Validation Loss:10.185/ F1_score:0.297/ Precision:0.486/ Recall:0.214
[2019/03/25 10:28:10] Epoch 10\Batch 18400\ Train Loss:10.074
[2019/03/25 10:28:39] Epoch 10\Batch 18600\ Train Loss:10.074
[2019/03/25 10:29:08] Epoch 10\Batch 18800\ Train Loss:10.074
[2019/03/25 10:29:37] Epoch 10\Batch 19000\ Train Loss:10.075
[2019/03/25 10:30:07] Epoch 10\Batch 19200\ Train Loss:10.075
[2019/03/25 10:30:35] Epoch 10\Batch 19400\ Train Loss:10.075
[2019/03/25 10:31:05] Epoch 10\Batch 19600\ Train Loss:10.074
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1589.0000009999999, 'TP': 1502.0000009999999, 'FN': 5497.000001}
[201

[2019/03/25 11:17:42] Epoch 11\Batch 12200\ Train Loss:10.063
[2019/03/25 11:18:11] Epoch 11\Batch 12400\ Train Loss:10.062
[2019/03/25 11:18:40] Epoch 11\Batch 12600\ Train Loss:10.061
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1578.0000009999999, 'TP': 1503.0000009999999, 'FN': 5496.000001}
[2019/03/25 11:19:26] Epoch 11/ Validation Loss:10.183/ F1_score:0.298/ Precision:0.488/ Recall:0.215
[2019/03/25 11:19:56] Epoch 11\Batch 12800\ Train Loss:10.061
[2019/03/25 11:20:25] Epoch 11\Batch 13000\ Train Loss:10.060
[2019/03/25 11:20:54] Epoch 11\Batch 13200\ Train Loss:10.060
[2019/03/25 11:21:23] Epoch 11\Batch 13400\ Train Loss:10.060
[2019/03/25 11:21:52] Epoch 11\Batch 13600\ Train Loss:10.060
[2019/03/25 11:22:21] Epoch 11\Batch 13800\ Train Loss:10.060
[2019/03/25 11:22:50] Epoch 11\Batch 14000\ Train Loss:10.060
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1578.0000009999999, 'TP': 1504.0000009999999, 'FN': 5495.000001}
[201

...

[2019/03/25 17:19:03] Epoch 16\Batch 18800\ Train Loss:10.037
[2019/03/25 17:19:32] Epoch 16\Batch 19000\ Train Loss:10.038
[2019/03/25 17:20:01] Epoch 16\Batch 19200\ Train Loss:10.038
[2019/03/25 17:20:30] Epoch 16\Batch 19400\ Train Loss:10.038
[2019/03/25 17:20:59] Epoch 16\Batch 19600\ Train Loss:10.038
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1579.0000009999999, 'TP': 1505.0000009999999, 'FN': 5494.000001}
[2019/03/25 17:21:45] Epoch 16/ Validation Loss:10.172/ F1_score:0.299/ Precision:0.488/ Recall:0.215
[2019/03/25 17:22:15] Epoch 16\Batch 19800\ Train Loss:10.038
[2019/03/25 17:22:44] Epoch 16\Batch 20000\ Train Loss:10.038
[2019/03/25 17:23:13] Epoch 16\Batch 20200\ Train Loss:10.038
[2019/03/25 17:23:42] Epoch 16\Batch 20400\ Train Loss:10.038
[2019/03/25 17:24:11] Epoch 16\Batch 20600\ Train Loss:10.038
[2019/03/25 17:24:40] Epoch 16\Batch 20800\ Train Loss:10.038
[2019/03/25 17:25:09] Epoch 16\Batch 21000\ Train Loss:10.038
run model on vali

[2019/03/25 18:10:45] Epoch 17\Batch 13200\ Train Loss:10.035
[2019/03/25 18:11:14] Epoch 17\Batch 13400\ Train Loss:10.035
[2019/03/25 18:11:43] Epoch 17\Batch 13600\ Train Loss:10.035
[2019/03/25 18:12:12] Epoch 17\Batch 13800\ Train Loss:10.035
[2019/03/25 18:12:41] Epoch 17\Batch 14000\ Train Loss:10.035
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1579.0000009999999, 'TP': 1506.0000009999999, 'FN': 5493.000001}
[2019/03/25 18:13:28] Epoch 17/ Validation Loss:10.172/ F1_score:0.299/ Precision:0.488/ Recall:0.215
[2019/03/25 18:13:57] Epoch 17\Batch 14200\ Train Loss:10.035
[2019/03/25 18:14:26] Epoch 17\Batch 14400\ Train Loss:10.035
[2019/03/25 18:14:55] Epoch 17\Batch 14600\ Train Loss:10.035
[2019/03/25 18:15:24] Epoch 17\Batch 14800\ Train Loss:10.035
[2019/03/25 18:15:53] Epoch 17\Batch 15000\ Train Loss:10.036
[2019/03/25 18:16:22] Epoch 17\Batch 15200\ Train Loss:10.036
[2019/03/25 18:16:51] Epoch 17\Batch 15400\ Train Loss:10.036
run model on vali

[2019/03/25 19:02:30] Epoch 18\Batch 7600\ Train Loss:10.041
[2019/03/25 19:02:59] Epoch 18\Batch 7800\ Train Loss:10.041
[2019/03/25 19:03:28] Epoch 18\Batch 8000\ Train Loss:10.040
[2019/03/25 19:03:57] Epoch 18\Batch 8200\ Train Loss:10.040
[2019/03/25 19:04:26] Epoch 18\Batch 8400\ Train Loss:10.040
run model on validation data...
label: [572, 793] predict: [572]
{'FP': 1579.0000009999999, 'TP': 1505.0000009999999, 'FN': 5494.000001}
[2019/03/25 19:05:13] Epoch 18/ Validation Loss:10.171/ F1_score:0.299/ Precision:0.488/ Recall:0.215
[2019/03/25 19:05:42] Epoch 18\Batch 8600\ Train Loss:10.040
[2019/03/25 19:06:11] Epoch 18\Batch 8800\ Train Loss:10.039
[2019/03/25 19:06:40] Epoch 18\Batch 9000\ Train Loss:10.038
[2019/03/25 19:07:09] Epoch 18\Batch 9200\ Train Loss:10.037
[2019/03/25 19:07:38] Epoch 18\Batch 9400\ Train Loss:10.038
[2019/03/25 19:08:07] Epoch 18\Batch 9600\ Train Loss:10.039
[2019/03/25 19:08:36] Epoch 18\Batch 9800\ Train Loss:10.040
run model on validation data.

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
